<a href="https://colab.research.google.com/github/trngbro/AI/blob/main/Progress20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

init